*Hello and welcome!* 👋

This notebook is the <u>fourth and last</u> part of a **tutorial** on how to **collect data from Twitter API v2 using Python** 🤓

In this notebook, we will briefly explore the filtered stream of tweets endpoints which allows us to collect tweets in real time.

### Importing packages and loading credentials
We start by importing the necessary packages to run the code.

In [ ]:
import requests
import json
import time
import random
import os

We import our *bearer_token* which we previously defined as an environment variable. This way you do not have to expose your credentials in your code.

In [ ]:
bearer_token = os.environ.get("BEARER_TOKEN")

### Authentication
Authentication is done by bearer token.

In [ ]:
def request_headers(bearer_token: str) -> dict:
    """
    Set up the request headers. 
    Returns a dictionary summarising the bearer token authentication details.

    Args:
        bearer_token: bearer token credentials
    """
    return {"Authorization": "Bearer {}".format(bearer_token)}

In [ ]:
headers = request_headers(bearer_token)

### Function to get rules in stream


In [ ]:
def get_rules(headers: dict):
    """
    Gets current rules in stream.
    Args:
        headers: request headers
    """
    rules_endpoint ="https://api.twitter.com/2/tweets/search/stream/rules"
    response = requests.request("GET",
        url = rules_endpoint, headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    return response.json()

At the moment we have no rules in the stream:

In [ ]:
rules = get_rules(headers)

In [ ]:
rules

### Function to add rules to stream


In [ ]:
def add_rules_to_stream(rules: list):
    """
    Adds a set of rules to the stream.
    Args:
        rules: list of rules to add
    """
    rules_to_add = {"add": rules}
    endpoint_url = "https://api.twitter.com/2/tweets/search/stream/rules"
    response = requests.request("POST",
        url = endpoint_url,
        headers=headers,
        json=rules_to_add,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print("The following rules we added to our stream:")
    print(response.json())

In [ ]:
sample_rules = [
        {"value": "#chatgpt or chatgpt", "tag": "chat gpt"},
        {"value": "heat pumps", "tag": "heat pumps"},
    ]


In [ ]:
add_rules_to_stream(sample_rules)

### Delete rules from stream


In [ ]:
def delete_rules_from_stream(ids:list, headers:dict):
    """
    Deletes a set of rules from the stream:
    Args:
        ids: ids from rules to delete
        headers: request headers
    """
    ids_to_delete = {"delete": {"ids": ids}}
    
    rules_endpoint = "https://api.twitter.com/2/tweets/search/stream/rules"
    response = requests.request("POST",
        url = rules_endpoint,
        headers=headers,
        json=ids_to_delete
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print("Summary:")
    print(response.json())

In [ ]:
delete_rules_from_stream(["1613120373439987714"], headers)

In [ ]:
rules = get_rules(headers)

In [ ]:
rules

### Getting stream of tweets

In [ ]:
def get_stream_of_tweets():
    """
    Connects to filtered stream endpoint and prints stream of tweets.
    """
    endpoint_url = "https://api.twitter.com/2/tweets/search/stream"
    response = requests.get(
        url = endpoint_url, headers=headers, stream=True,
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
get_stream_of_tweets()